# Partie 1: extraction des mots clés

### Imports

In [ ]:
import os
import yake

### Extraction des mots clés de notre corpus CAMille avec Yake

In [ ]:
# Instantier l'extracteur de mots clés
kw_extractor = yake.KeywordExtractor(lan="fr", top=50)
kw_extractor

In [ ]:
# Liste des Fichiers
data_path = "../data/txt/"
files = os.listdir(data_path)

In [ ]:
# Impression du nombre de fichiers identifiés
len(files)

### L'année que nous avons sélectionné est 1969

In [ ]:
# Choix des fichiers de l'année sélectionnée
this_file = [f for f in files if '1969' in f]
this_file

### Nombre de fichiers total sélectionnés: 

In [ ]:
#afficher le nombre de fichiers de cette année
len(this_file)

In [ ]:
# Récupération des textes de tous les fichiers de cette année et impression des 1000 premiers mots du 1er fichier
text = "" 
for f in this_file: file_path = os.path.join(data_path, f) 
with open(file_path, 'r', encoding='utf-8') as f:

    text += f.read() 
    print (text[:1000])

## Extraction des mots clés et conservation des bigrammes

In [ ]:
for f in sorted(this_file):
    text = open(os.path.join(data_path, f), 'r', encoding="utf-8").read()
    keywords = kw_extractor.extract_keywords(text)
    kept = []
    for kw, score in keywords:
        words = kw.split()
        if len(words) == 2:
            kept.append(kw)
    print(f"{f} mentions these keywords: {', '.join(kept)}...")

## Partie 2: enrichissement des stopwords et nuage de points

### Import 

In [ ]:
from collections import Counter
from wordcloud import WordCloud
import os
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from IPython.display import Image

### stopwords enrichis 

In [ ]:
# Stopwords (Idem que dans s1)
sw = stopwords.words("french")
sw += ["les", "plus", "cette", "fait", "faire", "être", "deux", "comme", "dont", "tout",
       "ils", "bien", "sans", "peut", "tous", "après", "ainsi", "donc", "cet", "sous",
       "celle", "entre", "encore", "toutes", "pendant", "moins", "dire", "cela", "non",
       "faut", "trois", "aussi", "dit", "avoir", "doit", "contre", "depuis", "autres",
       "van", "het", "autre", "jusqu", "ville", "rossel", "dem",
       "avant", "leurs", "suite", "très", "bruxelles", "belgique", "soir", "un", "d'",
       "gand", "près", "bon", "anvers", "mercredi", "louer", "octobre", "maison", "étage", "peu",
       "jardin", "anderlecht" , "tél", "rue", "appart", "cuis", "gar", "prix", "app", "ans", "pré", "brux", "dès" ]
sw = set(sw)

### Création du fichier contenant le texte de tous les journaux de l'année 1969

In [ ]:
content_list = []
for f in this_file:
    with open(os.path.join(data_path, f), 'r', encoding='utf-8') as f:
        content_list.append(f.read())

In [ ]:
#Compter le nombre d'éléments dans la liste
len(content_list)

In [ ]:
# Imprimer les 200 premiers caractères du contenu du premier fichier
content_list[0][0:200]

In [ ]:
# Ecrire tout le contenu dans un fichier temporaire
year = 1969
temp_path = '../data/tmp'
if not os.path.exists(temp_path):
    os.mkdir(temp_path)
with open(os.path.join(temp_path, f'{year}.txt'), 'w', encoding='utf-8') as f:
    f.write(' '.join(content_list))

In [ ]:
# Imprimer le contenu du fichier et constater les "déchets"
with open(os.path.join(temp_path, f'{year}.txt'), 'r', encoding='utf-8') as f:
    before = f.read()

before[:500]

### Nettoyage du fichier à l'aide d'une fonction de nettoyage

In [ ]:
def clean_text(year, folder=None):
    if folder is None:
        input_path = f"{year}.txt"
        output_path = f"{year}_clean.txt"
    else:
        input_path = f"{folder}/{year}.txt"
        output_path = f"{folder}/{year}_clean.txt"
    output = open(output_path, "w", encoding='utf-8')
    with open(input_path, 'r', encoding='utf-8') as f:
        text = f.read()
        words = nltk.wordpunct_tokenize(text)
        kept = [w.upper() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
        kept_string = " ".join(kept)
        output.write(kept_string)
    return f'Output has been written in {output_path}!'

### Application de la fonction sur le fichier complet de l'année

In [ ]:
clean_text(year, folder=temp_path)

In [ ]:
# Résultat
with open(os.path.join(temp_path, f'{year}_clean.txt'), 'r', encoding='utf-8') as f:
    after = f.read()

after[:500]

## Nuages de mots

In [ ]:
#AFFICHAGE DES TERMES LES PLUS FREQUENTS
frequencies = Counter(after.split())
print(frequencies.most_common(10))

### Création, stockage et affichage du nuage de mots

In [ ]:
cloud = WordCloud(width=2000, height=1000, background_color='white').generate_from_frequencies(frequencies)
cloud.to_file(os.path.join(temp_path, f"{year}.png"))
Image(filename=os.path.join(temp_path, f"{year}.png"))

## Partie 3 : reconnaissance d'entités nommées

### Imports 

In [ ]:
from collections import defaultdict
import sys
import spacy
from spacy.lang.fr.examples import sentences

In [ ]:
nlp = spacy.load('fr_core_news_md')

## Traitement du texte

Chargement du texte

In [ ]:
import os
data_path = "../data/txt/"
files = os.listdir(data_path)
this_file = [f for f in files if '1969' in f]
this_file
text = "" 
for f in this_file: file_path = os.path.join(data_path, f) 
with open(file_path, 'r', encoding='utf-8') as f:

    text += f.read() 

In [ ]:
%%time
# Traiter le texte

doc = nlp(text)

## Reconnaissance d'entité type ( PER)

In [ ]:
# Compter les entités
people = defaultdict(int)
for ent in doc.ents:
    if ent.label_ == "PER" and len(ent.text) > 3:
        people[ent.text] += 1

In [ ]:
# Trier et imprimer

sorted_people = sorted(people.items(), key=lambda kv: kv[1], reverse=True)

for person, freq in sorted_people[:50]:
    print(f"{person} apparait {freq} fois dans le corpus")

## Reconnaissance d'entité type (LOC)

In [ ]:
location = defaultdict(int)
for ent in doc.ents:
    if ent.label_ == "LOC" and len(ent.text) > 3:
        location[ent.text] += 1
        
        sorted_location = sorted(location.items(), key=lambda kv: kv[1], reverse=True)

for location, freq in sorted_location [:50]:
    print(f"{location} apparait {freq} fois dans le corpus")

## Reconnaissance d'entité type (ORG)

In [ ]:
organisation = defaultdict(int)
for ent in doc.ents:
    if ent.label_ == "ORG" and len(ent.text) > 3:
        organisation[ent.text] += 1
        
        sorted_organisation = sorted(organisation.items(), key=lambda kv: kv[1], reverse=True)

for organisation, freq in sorted_organisation [:50]:
    print(f"{organisation} apparait {freq} fois dans le corpus")

## Partie 4 : analyse des sentiments

### Imports

In [ ]:
import sys
from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer

### Création de la fonction get_sentiment

In [ ]:
tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

def get_sentiment(input_text):
    blob = tb(input_text)
    polarity, subjectivity = blob.sentiment
    polarity_perc = f"{100*abs(polarity):.0f}"
    subjectivity_perc = f"{100*subjectivity:.0f}"
    if polarity > 0:
        polarity_str = f"{polarity_perc}% positive"
    elif polarity < 0:
        polarity_str = f"{polarity_perc}% negative"
    else:
        polarity_str = "neutral"
    if subjectivity > 0:
        subjectivity_str = f"{subjectivity}% subjective"
    else:
        subjectivity_str = "perfectly objective"
    print(f"This text is {polarity_str} and {subjectivity_str}.")

#Sélection des 10 phrases
-	Le match d’Ostende constituera, en tout cas, la tête d’affiche de ces quarts de finale tirés samedi midi. 
-	Les autres rencontres paraissent moins indécises, en effet L’avantage semblant demeurer  à chaque fois aux équipes visitées.
-	Cette rencontre se disputera sans doute le mardi 22 pour éviter la concurrence éventuelle de la demi-finale de la Coupe d’Europe.
-	Le championnat est loin d’être terminé, mais un fait se dégage de la journée qui a donné une leçon d’efficacité.
-	Dans le bas du classement, mauvaise journée pour le C.S.
-	Il n'a pas joué avec la conviction attendue pour une équipe menacée
-	Saint- Nicolas, assez faible, a été battu à pied comme on s’y attendait
-	Ainsi Verviers a un retard deux points qu'il ne parviendra, selon- toute vraisemblance, que très difficilement à combler.
-	Il ne s’agira cependant pas d'une déchéance pour le capitaine brugeois
-	Débutant sous une averse de neige, la partie prit immédiatement une tournure favorable pour le F.C. Liégeois

### Analyse des sentiments de chacune des phrases

In [ ]:
get_sentiment("Le match d’Ostende constituera, en tout cas, la tête d’affiche de ces quarts de finale tirés samedi midi.")

In [ ]:
get_sentiment("Les autres rencontres paraissent moins indécises, en effet L’avantage semblant demeurer  à chaque fois aux équipes visitées.")

In [ ]:
get_sentiment("Cette rencontre se disputera sans doute le mardi 22 pour éviter la concurrence éventuelle de la demi-finale de la Coupe d’Europe.")

In [ ]:
get_sentiment("Le championnat est loin d’être terminé, mais un fait se dégage de la journée qui a donné une leçon d’efficacité.")

In [ ]:
get_sentiment("Dans le bas du classement, mauvaise journée pour le C.S.")

In [ ]:
get_sentiment("Il n'a pas joué avec la conviction attendue pour une équipe menacée")

In [ ]:
get_sentiment("Saint- Nicolas, assez faible, a été battu à pied comme on s’y attendait")

In [ ]:
get_sentiment("Ainsi Verviers a un retard deux points qu'il ne parviendra, selon- toute vraisemblance, que très difficilement à combler.")

In [ ]:
get_sentiment("Il ne s’agira cependant pas d'une déchéance pour le capitaine brugeois")

In [ ]:
get_sentiment("Débutant sous une averse de neige, la partie prit immédiatement une tournure favorable pour le F.C. Liégeois")

### Utilisation des transformers 

### Imports

In [ ]:
!pip install tensorflow
!pip install sentencepiece
!pip install transformers

from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import pipeline

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("tblard/tf-allocine", use_pt=True)
model = TFAutoModelForSequenceClassification.from_pretrained("tblard/tf-allocine")

sentiment_analyser = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

### Analyse du sentiment de la phrase

In [ ]:
sentiment_analyser("Le match d’Ostende constituera, en tout cas, la tête d’affiche de ces quarts de finale tirés samedi midi.")

In [ ]:
sentiment_analyser("Les autres rencontres paraissent moins indécises, en effet L’avantage semblant demeurer  à chaque fois aux équipes visitées.")

In [ ]:
sentiment_analyser("Cette rencontre se disputera sans doute le mardi 22 pour éviter la concurrence éventuelle de la demi-finale de la Coupe d’Europe.")

In [ ]:
sentiment_analyser("Le championnat est loin d’être terminé, mais un fait se dégage de la journée qui a donné une leçon d’efficacité.")

In [ ]:
sentiment_analyser("Dans le bas du classement, mauvaise journée pour le C.S.")

In [ ]:
sentiment_analyser("Il n'a pas joué avec la conviction attendue pour une équipe menacée")

In [ ]:
sentiment_analyser("Saint- Nicolas, assez faible, a été battu à pied comme on s’y attendait")

In [ ]:
sentiment_analyser("Ainsi Verviers a un retard deux points qu'il ne parviendra, selon- toute vraisemblance, que très difficilement à combler.")

In [ ]:
sentiment_analyser("Il ne s’agira cependant pas d'une déchéance pour le capitaine brugeois")

In [ ]:
sentiment_analyser("Débutant sous une averse de neige, la partie prit immédiatement une tournure favorable pour le F.C. Liégeois")